In [1]:
import bugzilla
bzapi = bugzilla.Bugzilla('bugzilla.redhat.com')
query = bzapi.build_query(product='Fedora')
query['blocks'] = 1890881  # PYTHON3.10
bugz = bzapi.query(query)

In [2]:
F35FailsToInstall = 1927313
F35FTBFS = 1927309

In [3]:
todo = !cat todo.pkgs

In [4]:
bugz = [b for b in bugz if b.status != 'CLOSED']

In [5]:
len(bugz)

535

In [6]:
to_block_ftbfs = set()
for component in todo:
    our_bugz = [b for b in bugz if b.component == component]
    if (num_bugz := len(our_bugz)) > 2:
        print(f"{component} has {num_bugz} bugz:")
        for bug in our_bugz:
            print(f"  https://bugzilla.redhat.com/{bug.id}")
    elif num_bugz == 2:
        fti_bugz = [b for b in our_bugz if F35FailsToInstall in b.blocks]
        other_bugz = [b for b in our_bugz if F35FailsToInstall not in b.blocks]
        if len(fti_bugz) == 1:
            fti_bug = fti_bugz[0]
            ftbs_bug = other_bugz[0]
            print(f"{component} {fti_bug.id} DUPLICATE of {ftbs_bug.id}")
            fti_bug.close('DUPLICATE', dupeid=ftbs_bug.id,
                         comment='Automatically marking as duplicate. Please forgive me if not accurate.')
            to_block_ftbfs.add(ftbs_bug.id)
        else:
            print(f"{component} has 2 bugz, but in an unexpected state:")
            for bug in our_bugz:
                print(f"  https://bugzilla.redhat.com/{bug.id}")

sagemath 1969187 DUPLICATE of 1964990
sshuttle 1969190 DUPLICATE of 1949447
sugar-datastore 1969191 DUPLICATE of 1900789
trademgen 1969196 DUPLICATE of 1898929
watchman 1969200 DUPLICATE of 1965963


In [7]:
to_block_ftbfs

{1898929, 1900789, 1949447, 1964990, 1965963}

In [8]:
block_f35ftbfs = bzapi.build_update(blocks_add=F35FTBFS)
block_f35ftbfs["minor_update"] = True
bzapi.update_bugs(sorted(to_block_ftbfs), block_f35ftbfs)

{'bugs': [{'changes': {'blocks': {'removed': '', 'added': '1927309'}},
   'last_change_time': <DateTime '20210608T11:50:21' at 0x7fbc7419c280>,
   'id': 1898929,
   'alias': []},
  {'changes': {'blocks': {'removed': '', 'added': '1927309'}},
   'last_change_time': <DateTime '20210608T11:50:21' at 0x7fbc7419c430>,
   'id': 1900789,
   'alias': []},
  {'changes': {'blocks': {'removed': '', 'added': '1927309'}},
   'last_change_time': <DateTime '20210608T11:50:21' at 0x7fbc7419c2b0>,
   'id': 1949447,
   'alias': []},
  {'changes': {},
   'last_change_time': <DateTime '20210608T11:49:31' at 0x7fbc7419c0d0>,
   'id': 1964990,
   'alias': []},
  {'changes': {},
   'last_change_time': <DateTime '20210608T11:50:09' at 0x7fbc7419c790>,
   'id': 1965963,
   'alias': []}]}